In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
     |████████████████████████████████| 276kB 2.6MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44074 sha256=5a7ea88949cfe61b74e4a081c30f6e3f943de478763be9c27c76e3df3301135b
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-2h_e4z2l
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-2h_e4z2l
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-lin

In [ ]:
%cd /content
!rm -rf detectron2/
!git clone https://github.com/chwchao/detectron2.git

/content
Cloning into 'detectron2'...
remote: Enumerating objects: 5121, done.
remote: Total 5121 (delta 0), reused 0 (delta 0), pack-reused 5121
Receiving objects: 100% (5121/5121), 2.47 MiB | 1.96 MiB/s, done.
Resolving deltas: 100% (3671/3671), done.


In [ ]:
# install detectron2
!python -m pip install -e detectron2

Obtaining file:///content/detectron2
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200616-cp36-none-any.whl size=41027 sha256=1571c00552d647d0c4078af3cc3dfc35e379ed4c0359bdf78f97358bdbb5f1c5
  Stored in directory: /root/.cache/pip/wheels/41/d8/fa/3ce58db3bff39667182416191bb442b206af1dbd9a5d0804cb
Successfully built fvcore
  Running setup.py develop for detectron2


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

model_final_f10217.pkl: 178MB [00:15, 11.3MB/s]                           


In [ ]:
%mkdir /content/dataset
%cd /content/dataset
%mkdir Train
%mkdir Dev

/content/dataset


In [ ]:
import os
import csv
from IPython.display import HTML, display

In [ ]:
# Progress bar
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
# Train data
%mkdir /content/Train
with open('/content/drive/My Drive/AI_Cup/C1-P1_Train Dev_fixed/train.csv', newline='') as originalFile:
  rows = csv.reader(originalFile)
  # Progress bar
  out = display(progress(0, len(list(rows))), display_id=True)
  progress_count = 0

  with open('/content/train.csv', 'w', newline='') as newFile:

    # CSV writer & write first row
    writer = csv.writer(newFile)
    writer.writerow(['image_id', 'label'])
    
    first = True
    for row in rows:
      print(progress_count)
      if first:
        first = False
        continue
      im = load_image("/content/drive/My Drive/AI_Cup/C1-P1_Train Dev_fixed/C1-P1_Train/" + row[0], target_size=(224,224,3))
      im = img_to_array(im)
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
      v, isfind = v.draw_instance_predictions_crop(outputs["instances"].to("cpu"), 47, 54, True)
      if isfind:
        v.save("/content/Train/" + row[0])
        writer.writerow(row)

      # Update progress bar
      progress_count = progress_count + 1
      out.update(progress(progress_count, len(list(rows))))

mkdir: cannot create directory ‘/content/Train’: File exists


0


In [ ]:
# Dev data
with open('/content/drive/My Drive/AI_Cup/C1-P1_Train Dev_fixed/dev.csv', newline='') as originalFile:
  rows = csv.reader(originalFile)

  with open('dev.csv', 'w', newline='') as newFile:

    # CSV writer & write first row
    writer = csv.writer(newFile)
    writer.writerow(['image_id', 'label'])

    first = True
    count = 0
    for row in rows:
      if first:
        first = False
        continue
      count = count + 1
      if count%100 == 0:
        print(count)
      im = cv2.imread("/content/drive/My Drive/AI_Cup/C1-P1_Train Dev_fixed/C1-P1_Dev/" + row[0])
      outputs = predictor(im)
      v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
      v, isfind = v.draw_instance_predictions_crop(outputs["instances"].to("cpu"), 47, 54, True)
      v.save("Dev/" + row[0])
      writer.writerow(row)


In [ ]:
!zip -r /content/datasets.zip /content/dataset

In [ ]:
%cp /content/datasets.zip /content/drive/My\ Drive/AI_Cup/